# 6-7. 프로젝트: 멋진 작사가 만들기

1. 가사 텍스트 생성 모델이 정상적으로 동작하는가? 텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?

2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?
특수문자 제거, 토크나이저 생성, 패딩 처리 등의 과정이 빠짐없이 진행되었는가?

3. 텍스트 생성모델이 안정적으로 학습되었는가?
텍스트 생성모델의 validation loss 가 2.2 이하로 낮아졌는가?


In [ ]:
# 라이브러리 버전 확인
!pip list | grep glob

import glob
import tensorflow

#print(glob.__version__)
print(tensorflow.__version__)

## Step 1. 데이터 다운로드
## Step 2. 데이터 읽어오기

glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요. glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할게요!

In [ ]:
import glob
import os

txt_file_path = '/content/drive/MyDrive/0.AIFFEL/LMS node/Exp 06/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

In [63]:
txt_list

['/content/drive/MyDrive/0.AIFFEL/LMS node/Exp 06/lyricist/data/lyrics/janisjoplin.txt',
 '/content/drive/MyDrive/0.AIFFEL/LMS node/Exp 06/lyricist/data/lyrics/nursery_rhymes.txt',
 '/content/drive/MyDrive/0.AIFFEL/LMS node/Exp 06/lyricist/data/lyrics/dickinson.txt',
 '/content/drive/MyDrive/0.AIFFEL/LMS node/Exp 06/lyricist/data/lyrics/nicki-minaj.txt',
 '/content/drive/MyDrive/0.AIFFEL/LMS node/Exp 06/lyricist/data/lyrics/disney.txt',
 '/content/drive/MyDrive/0.AIFFEL/LMS node/Exp 06/lyricist/data/lyrics/bob-marley.txt',
 '/content/drive/MyDrive/0.AIFFEL/LMS node/Exp 06/lyricist/data/lyrics/beatles.txt',
 '/content/drive/MyDrive/0.AIFFEL/LMS node/Exp 06/lyricist/data/lyrics/dj-khaled.txt',
 '/content/drive/MyDrive/0.AIFFEL/LMS node/Exp 06/lyricist/data/lyrics/kanye.txt',
 '/content/drive/MyDrive/0.AIFFEL/LMS node/Exp 06/lyricist/data/lyrics/patti-smith.txt',
 '/content/drive/MyDrive/0.AIFFEL/LMS node/Exp 06/lyricist/data/lyrics/drake.txt',
 '/content/drive/MyDrive/0.AIFFEL/LMS node/E

In [64]:
# 데이터 확인 

raw_corpus

['',
 "I've got a lot of work to do",
 "I ain't finished blowin' it, nice bra",
 'My slow flows remarkable',
 'Call up DJ Mano, shout out Twilite Tone',
 'Can I have a drink of water?',
 'Come on babe, come on, oh yeah',
 'You have the same passion that I have',
 "(Stop while it's not too late) And I still don't know Finally taking flight",
 'In the magazines',
 "I'm rich (I'm rich)",
 'Toronto',
 "I don't know but boy what's up (Throw it up, throw it up girl)",
 'Why we just speak at night',
 "When night comes and you're on your own",
 'NiXga creep up on your ass hahaha',
 'But I can be tonight',
 'I’m gazing out the window',
 'Little sparrow, little sparrow',
 "In every way I love you (In every way I'll help you?)",
 "And I admit that I'm feeling a little pressure, when you're telling me I better come get it",
 "That kush I promise that's my dude",
 'Depende da paixao -- It depends on the passion',
 'When I know that I could be more than you could ever dream',
 "There's so much more 

In [65]:
# 데이터 확인 결과 중복되는 것이 많아서 중복된 corpus 제거해줌.

raw_corpus = list(set(raw_corpus))

In [66]:
len(raw_corpus)

118168

## Step 3. 데이터 정제

- preprocess_sentence()


In [67]:
import re
# 입력된 문장들을 정제하는 작업

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [69]:
# 정제된 문장확인

print(preprocess_sentence(raw_corpus[4]))

<start> call up dj mano , shout out twilite tone <end>


In [70]:
# 토큰의 개수가 15개를 넘는 것은 제외 

corpus_15 = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split()) > 15: continue
    corpus_15.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus_15[:10]

['<start> i ve got a lot of work to do <end>',
 '<start> i ain t finished blowin it , nice bra <end>',
 '<start> my slow flows remarkable <end>',
 '<start> call up dj mano , shout out twilite tone <end>',
 '<start> can i have a drink of water ? <end>',
 '<start> come on babe , come on , oh yeah <end>',
 '<start> you have the same passion that i have <end>',
 '<start> in the magazines <end>',
 '<start> toronto <end>',
 '<start> i m rich i m rich <end>']

In [71]:
len(corpus_15)

102070

In [72]:
import tensorflow as tf
# tensorflow tokenizer와 pad_sequences를 사용한 토큰화

def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus_15)

[[   2    5   89 ...    0    0    0]
 [   2    5   96 ...    0    0    0]
 [   2   13  431 ...    0    0    0]
 ...
 [   2   26  237 ...    3    0    0]
 [   2  214   26 ...  134    3    0]
 [   2  489 1524 ...  257    3    0]] <keras_preprocessing.text.Tokenizer object at 0x7f249f572e50>


In [73]:
tensor.shape

(102070, 15)

In [74]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2   5  89  40   9 417  18 244  10  48   3   0   0   0]
[  5  89  40   9 417  18 244  10  48   3   0   0   0   0]


## Step 4. 평가 데이터셋 분리

훈련 데이터와 평가 데이터를 분리하세요!

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상 으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!

In [75]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=9082)

In [77]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>

In [88]:
print('Source Train: ', enc_train.shape)
print('Target validation: ', dec_val.shape)

Source Train:  (81656, 14)
Target validation:  (20414, 14)


## Step 5. 인공지능 만들기

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

In [90]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [91]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.07534615e-04, -4.07259504e-04,  1.21410885e-04, ...,
          4.09677101e-04,  6.26270019e-04,  4.94511158e-04],
        [-2.53939594e-04, -8.01751565e-04, -5.67470579e-06, ...,
          5.31236990e-04,  5.90050418e-04,  6.29279763e-04],
        [-2.72049772e-04, -1.28175947e-03, -2.45643128e-06, ...,
          3.01688211e-04,  5.68389296e-05,  8.83076922e-04],
        ...,
        [ 4.44802601e-04, -3.33976524e-04, -2.46398384e-03, ...,
          5.60539775e-04,  2.20452342e-03, -1.88891450e-03],
        [ 6.82569284e-04, -2.20018381e-04, -2.58011208e-03, ...,
          9.06432571e-04,  2.30692723e-03, -2.09894241e-03],
        [ 8.27933254e-04, -4.59431540e-05, -2.47553852e-03, ...,
          1.16314739e-03,  2.28278246e-03, -2.07980792e-03]],

       [[ 1.07534615e-04, -4.07259504e-04,  1.21410885e-04, ...,
          4.09677101e-04,  6.26270019e-04,  4.94511158e-04],
        [ 3.99164186e-04, -3.69706802e-04, -3

In [92]:
model.summary()

Model: "text_generator_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     multiple                  12289024  
                                                                 
 lstm_14 (LSTM)              multiple                  25174016  
                                                                 
 lstm_15 (LSTM)              multiple                  33562624  
                                                                 
 dense_7 (Dense)             multiple                  24590049  
                                                                 
Total params: 95,615,713
Trainable params: 95,615,713
Non-trainable params: 0
_________________________________________________________________


In [94]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


model.compile(loss=loss, optimizer=optimizer)


In [95]:
# early stop
es = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

# 모델 훈련

model.fit(dataset, epochs=39, validation_data = (enc_val,dec_val),callbacks=[es])

Epoch 1/39
318/318 [==============================] - 93s 287ms/step - loss: 3.5022 - val_loss: 3.1242
Epoch 2/39
318/318 [==============================] - 91s 285ms/step - loss: 2.9885 - val_loss: 2.9694
Epoch 3/39
318/318 [==============================] - 91s 285ms/step - loss: 2.8025 - val_loss: 2.8945
Epoch 4/39
318/318 [==============================] - 91s 285ms/step - loss: 2.6305 - val_loss: 2.8603
Epoch 5/39
318/318 [==============================] - 91s 285ms/step - loss: 2.4457 - val_loss: 2.8506
Epoch 6/39
318/318 [==============================] - 91s 284ms/step - loss: 2.2417 - val_loss: 2.8600
Epoch 7/39
318/318 [==============================] - 91s 285ms/step - loss: 2.0192 - val_loss: 2.8876
Epoch 7: early stopping


In [96]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [97]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so i can t stand <end> '

In [98]:
generate_text(model, tokenizer, init_sentence="<start> i wanna", max_len=20)

'<start> i wanna be your lover partyup controversy <end> '

In [99]:
generate_text(model, tokenizer, init_sentence="<start> what a", max_len=20)

'<start> what a lovely creation <end> '

In [100]:
generate_text(model, tokenizer, init_sentence="<start> I wish you", max_len=20)

'<start> i wish you would nigga <end> '

In [101]:
generate_text(model, tokenizer, init_sentence="<start> As time", max_len=20)

'<start> as time cannot decay <end> '

In [102]:
generate_text(model, tokenizer, init_sentence="<start> life is ", max_len=20)

'<start> life is a <unk> , <end> '

In [103]:
generate_text(model, tokenizer, init_sentence="<start> You are ", max_len=20)

'<start> you are the one that i adore <end> '

In [104]:
generate_text(model, tokenizer, init_sentence="<start> Time to ", max_len=20)

'<start> time to see <end> '

In [105]:
generate_text(model, tokenizer, init_sentence="<start> This is the way", max_len=20)

'<start> this is the way you talk <end> '

In [106]:
generate_text(model, tokenizer, init_sentence="<start> When I was", max_len=20)

'<start> when i was your man <end> '

## 결과

- 배치사이즈 256으로 고정하고 (embedding_size, hidden_size) 를 (256,512), (512,1024), (1024,2048)로 바꿔가면서 학습을 진행했다. 예상했던 것은 embedding size와 hidden size 를 증가시킬수록 loss 가 줄어들것이라고 생각했는데, 시간은 숫자가 커지면서 오래걸린 것에 비해서 loss 는 2.8 근처에서 낮아지지 않았다. 


In [113]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 128
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(128, 14), dtype=tf.int32, name=None), TensorSpec(shape=(128, 14), dtype=tf.int32, name=None))>

In [114]:
embedding_size = 1024
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [115]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


model.compile(loss=loss, optimizer=optimizer)


In [116]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# early stop
es = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

# 모델 훈련

model.fit(dataset, epochs=39, validation_data = (enc_val,dec_val),callbacks=[es])

Epoch 1/39
637/637 [==============================] - 115s 158ms/step - loss: 3.3084 - val_loss: 3.0269
Epoch 2/39
637/637 [==============================] - 100s 157ms/step - loss: 2.8835 - val_loss: 2.8997
Epoch 3/39
637/637 [==============================] - 100s 157ms/step - loss: 2.6693 - val_loss: 2.8460
Epoch 4/39
637/637 [==============================] - 100s 158ms/step - loss: 2.4394 - val_loss: 2.8323
Epoch 5/39
637/637 [==============================] - 101s 158ms/step - loss: 2.1764 - val_loss: 2.8563
Epoch 6/39
637/637 [==============================] - 101s 158ms/step - loss: 1.8919 - val_loss: 2.9051
Epoch 6: early stopping


In [117]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so , i love you so <end> '

In [118]:
generate_text(model, tokenizer, init_sentence="<start> i wanna", max_len=20)

'<start> i wanna be your g . u . y . <end> '

In [119]:
generate_text(model, tokenizer, init_sentence="<start> what a", max_len=20)

'<start> what a perfect creation <end> '

In [120]:
generate_text(model, tokenizer, init_sentence="<start> I wish you", max_len=20)

'<start> i wish you would nigga <end> '

In [121]:
generate_text(model, tokenizer, init_sentence="<start> As time", max_len=20)

'<start> as time cannot decay <end> '

## 결과

- 배치사이즈 128으로 고정하고 (embedding_size, hidden_size) 를 (1024,2048)로 바꿔서 학습을 진행했다. 스터디중에 배치 사이즈에 대한 질문을 받고 작아지면 어떻게 될지 궁금해서 진행해보았으나 학습에는 큰 영향이 없는 것 같다. 

In [122]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 512
steps_per_epoch = len(enc_train) // BATCH_SIZEembedding_size = 2048
hidden_size = 3072
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


model.compile(loss=loss, optimizer=optimizer)


In [62]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# early stop
es = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

# 모델 훈련

model.fit(dataset, epochs=39, validation_data = (enc_val,dec_val),callbacks=[es])

Epoch 1/39
159/159 [==============================] - 214s 1s/step - loss: 3.6855 - val_loss: 3.2456
Epoch 2/39
159/159 [==============================] - 176s 1s/step - loss: 3.1200 - val_loss: 3.0554
Epoch 3/39
159/159 [==============================] - 176s 1s/step - loss: 2.9406 - val_loss: 2.9667
Epoch 4/39
159/159 [==============================] - 176s 1s/step - loss: 2.8044 - val_loss: 2.9206
Epoch 5/39
159/159 [==============================] - 176s 1s/step - loss: 2.6761 - val_loss: 2.8934
Epoch 6/39
159/159 [==============================] - 176s 1s/step - loss: 2.5449 - val_loss: 2.8824
Epoch 7/39
159/159 [==============================] - 176s 1s/step - loss: 2.4084 - val_loss: 2.8791
Epoch 8/39
159/159 [==============================] - 176s 1s/step - loss: 2.2625 - val_loss: 2.8924
Epoch 9/39
159/159 [==============================] - 176s 1s/step - loss: 2.1087 - val_loss: 2.9076
Epoch 9: early stopping


In [78]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you , baby <end> '

In [79]:
generate_text(model, tokenizer, init_sentence="<start> i wanna", max_len=20)

'<start> i wanna be your lover partyup controversy <end> '

In [80]:
generate_text(model, tokenizer, init_sentence="<start> what a", max_len=20)

'<start> what a shame <end> '

In [81]:
generate_text(model, tokenizer, init_sentence="<start> I wish you", max_len=20)

'<start> i wish you d never be mine <end> '

In [82]:
generate_text(model, tokenizer, init_sentence="<start> As time", max_len=20)

'<start> as time will be the <unk> <end> '

In [83]:
generate_text(model, tokenizer, init_sentence="<start> life is ", max_len=20)

'<start> life is a roller coaster <end> '

In [84]:
generate_text(model, tokenizer, init_sentence="<start> You are ", max_len=20)

'<start> you are the lady in my life doo doo doo <end> '

In [85]:
generate_text(model, tokenizer, init_sentence="<start> Time to ", max_len=20)

'<start> time to make you feel like a <unk> <end> '

In [86]:
generate_text(model, tokenizer, init_sentence="<start> This is the way", max_len=20)

'<start> this is the way you do <end> '

In [87]:
generate_text(model, tokenizer, init_sentence="<start> When I was", max_len=20)

'<start> when i was your man <end> '

## 결과

- 배치사이즈 512으로 고정하고 (embedding_size, hidden_size) 를 (256,512), (1024,2048) (2048, 3072), (512, 3072) 로 바꿔서 학습을 진행했다. 여전히 학습에는 큰 영향이 없는 것 같다. 

In [146]:
import glob
import os

txt_file_path = '/content/drive/MyDrive/0.AIFFEL/LMS node/Exp 06/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Busted flat in Baton Rouge, waitin' for a train", "And I's feelin' near as faded as my jeans", 'Bobby thumbed a diesel down, just before it rained']


In [147]:
# 토큰의 개수가 15개를 넘는 것은 제외 

corpus_15 = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split()) > 15: continue
    corpus_15.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus_15[:10]

['<start> busted flat in baton rouge , waitin for a train <end>',
 '<start> and i s feelin near as faded as my jeans <end>',
 '<start> bobby thumbed a diesel down , just before it rained <end>',
 '<start> i was playin soft while bobby sang the blues , yeah <end>',
 '<start> windshield wipers slappin time , i was holdin bobby s hand in mine <end>',
 '<start> you know , feelin good was good enough for me <end>',
 '<start> there bobby shared the secrets of my soul <end>',
 '<start> through all kinds of weather , through everything we done <end>',
 '<start> he s lookin for that home , and i hope he finds it <end>',
 '<start> nothin , that s all that bobby left me , yeah <end>']

In [148]:
tensor, tokenizer = tokenize(corpus_15)

[[   2 4009 1674 ...    0    0    0]
 [   2    8    4 ...    0    0    0]
 [   2  871 6826 ...    0    0    0]
 ...
 [   2   22   22 ...    3    0    0]
 [   2   22 4254 ...    0    0    0]
 [   2    3    0 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f21020efad0>


In [131]:
tensor.shape

(156013, 15)

In [149]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2 4009 1674   14    1 4486    5 1295   28    9  636    3    0    0]
[4009 1674   14    1 4486    5 1295   28    9  636    3    0    0    0]


In [133]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>

In [150]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=9082)

In [151]:
print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_val.shape)

Source Train:  (124810, 14)
Target Train:  (31203, 14)


In [152]:
embedding_size = 1024
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [153]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


model.compile(loss=loss, optimizer=optimizer)


In [154]:
es = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
model.fit(dataset, epochs=39, validation_data = (enc_val,dec_val),callbacks=[es])

Epoch 1/39
609/609 [==============================] - 184s 277ms/step - loss: 3.1413 - val_loss: 2.7101
Epoch 2/39
609/609 [==============================] - 168s 276ms/step - loss: 2.5800 - val_loss: 2.2780
Epoch 3/39
609/609 [==============================] - 168s 276ms/step - loss: 2.2110 - val_loss: 1.8965
Epoch 4/39
609/609 [==============================] - 168s 276ms/step - loss: 1.8676 - val_loss: 1.5670
Epoch 5/39
609/609 [==============================] - 168s 276ms/step - loss: 1.5697 - val_loss: 1.3147
Epoch 6/39
609/609 [==============================] - 168s 276ms/step - loss: 1.3368 - val_loss: 1.1430
Epoch 7/39
609/609 [==============================] - 168s 276ms/step - loss: 1.1708 - val_loss: 1.0329
Epoch 8/39
609/609 [==============================] - 168s 276ms/step - loss: 1.0677 - val_loss: 0.9727
Epoch 9/39
609/609 [==============================] - 168s 276ms/step - loss: 1.0130 - val_loss: 0.9426
Epoch 10/39
609/609 [==============================] - 168s 276m

KeyboardInterrupt: ignored

* 이미 loss 값이 충분히 낮아서 중단했다. 

In [155]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)


'<start> i love you more than he <end> '

In [157]:
generate_text(model, tokenizer, init_sentence="<start> i wanna", max_len=20)


'<start> i wanna be with you <end> '

In [158]:
generate_text(model, tokenizer, init_sentence="<start> what a", max_len=20)


'<start> what a wonderful phrase <end> '

In [160]:
generate_text(model, tokenizer, init_sentence="<start> I wish you", max_len=20)


'<start> i wish you d do it again <end> '

In [161]:
generate_text(model, tokenizer, init_sentence="<start> As time", max_len=20)


'<start> as time cannot decay <end> '

In [162]:

generate_text(model, tokenizer, init_sentence="<start> Life is", max_len=20)


'<start> life is a beach and these hoes trying to lay out <end> '

In [163]:
generate_text(model, tokenizer, init_sentence="<start> You are", max_len=20)


'<start> you are the lady in my life let me touch you girl <end> '

In [164]:
generate_text(model, tokenizer, init_sentence="<start> Time to", max_len=20)


'<start> time to giddy up <end> '

In [165]:
generate_text(model, tokenizer, init_sentence="<start> This is the way", max_len=20)


'<start> this is the way you do it <end> '

In [166]:
generate_text(model, tokenizer, init_sentence="<start> When I was", max_len=20)

'<start> when i was a child i grew up by the river lea <end> '

In [167]:
# 오타가 있을 때 
generate_text(model, tokenizer, init_sentence="<start> ii lovd", max_len=20)


'<start> ii <unk> s , the <unk> s , the <unk> s <end> '

## 결과

- 여러번의 테스트를 거쳤음에도 오차가 거의 변화가 없었다. 텍스트 전처리 과정에서 중복되는 문장들이 많길래 set 함수를 써서 중복된 문장을 제거했는데 이것이 혹시 영향을 주었나 싶어서 중복된 문장들을 포함하고 학습을 했더니 hidden 과 embedding 크기 조절만으로 val-loss 값이 1이하로 낮아졌다. 




In [168]:
embedding_size = 256
hidden_size = 512
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


model.compile(loss=loss, optimizer=optimizer)

model.fit(dataset, epochs=39, validation_data = (enc_val,dec_val),callbacks=[es])

Epoch 1/39
609/609 [==============================] - 35s 51ms/step - loss: 3.6878 - val_loss: 3.2492
Epoch 2/39
609/609 [==============================] - 30s 50ms/step - loss: 3.1256 - val_loss: 3.0194
Epoch 3/39
609/609 [==============================] - 30s 50ms/step - loss: 2.9665 - val_loss: 2.8968
Epoch 4/39
609/609 [==============================] - 30s 49ms/step - loss: 2.8564 - val_loss: 2.7928
Epoch 5/39
609/609 [==============================] - 30s 49ms/step - loss: 2.7653 - val_loss: 2.7047
Epoch 6/39
609/609 [==============================] - 30s 49ms/step - loss: 2.6851 - val_loss: 2.6244
Epoch 7/39
609/609 [==============================] - 30s 49ms/step - loss: 2.6101 - val_loss: 2.5492
Epoch 8/39
609/609 [==============================] - 30s 50ms/step - loss: 2.5379 - val_loss: 2.4745
Epoch 9/39
609/609 [==============================] - 30s 49ms/step - loss: 2.4684 - val_loss: 2.4055
Epoch 10/39
609/609 [==============================] - 30s 50ms/step - loss: 2.402

KeyboardInterrupt: ignored

In [172]:
generate_text(model, tokenizer, init_sentence="<start> I wish", max_len=20)


'<start> i wish i could be the same <end> '

## 회고 

- 문장 자체가 같은 중복된 문장이 학습에 영향을 끼치지 않을거라고 생각했는데, 결과를 보고 다시 생각해보니 중복된 문장 자체도 입력 데이터로 들어가서 학습에 영향을 줄 수도 있을 것 같다. 
- 중복 문장을 뺀 데이터는 size를 크게 하지 않아도... loss 가 적게 나온다. 왜지...?
- val-loss 값 자체는 상당히 낮아졌는데, 나온 문장의 퀄리티는 그렇게 좋지 않은 것 같다. 과연 무슨 기준으로 문장이 잘 만들어졌는지 판단할 수 있을까라는 생각을 하게 되었다. val-loss 2.8 문장보다는 말이 되기는 하지만 여전히 어색한 것들도 있다.
- 오타나 학습에 들어가지 않았을 단어를 넣어보니 역시나 이상하게 나와서 이 부분은 재미있었다. 
